# Pipeline

In [ ]:
#ETL pipe
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString)

from sagemaker.workflow.functions import Join
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.pipeline import Pipeline

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

bucket = session.default_bucket()
prefix = '1_ins_dataset'
pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance Co Example"  # Model name in model registry
framework_version = "0.23-1"

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")

input_uri = Join(on="/", values=['s3://{}'.format(bucket),
                                      prefix,
                                      'raw'])

tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="ins-example-job"
)

step_etl = ProcessingStep(
    name="etl",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_uri, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(
            output_name="clean",
            source="/opt/ml/processing/output",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    'clean'
                ]
            )
        )
    ],
    code="etl.py"
)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count
    ],
    steps=[step_etl])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(
    execution_display_name="InsClean")

In [ ]:
#Preprocess pipe
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

from sagemaker.workflow.pipeline import Pipeline

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

bucket = session.default_bucket()
prefix = '1_ins_dataset'
pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance Co Example"  # Model name in model registry
framework_version = "0.23-1"

input_uri = f's3://{bucket}/{prefix}/clean/full_data.csv'


tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")
    
input_data = ParameterString(
    name="InputData",
    default_value=input_uri
)

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="ins-example-job"
)

step_preprocess = ProcessingStep(
    name="preprocess_data",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/output/train",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    'final',
                    "train"
                ],
            ),
        ),
        ProcessingOutput(
            output_name="validate",
            source="/opt/ml/processing/output/validate",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    'final',
                    "validate"
                ],
            ),
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/output/test",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    'final',
                    "test"
                ],
            ),
        ),
        ProcessingOutput(
            output_name="encoder",
            source="/opt/ml/processing/output/encoder",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    'final',
                    'encoder'
                ],
            ),
        ),
    ],
    code="preprocess.py"
)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        input_data,
    ],
    steps=[step_preprocess])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(execution_display_name="InsPreprocess10")

In [ ]:
#Train Prebuilt model
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.functions import Join
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.workflow.pipeline import Pipeline

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
prefix = '1_ins_dataset'

pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance Co Example"  # Model name in model registry
framework_version = "0.23-1"

train_uri = f's3://{bucket}/{prefix}/final/train/train_feats.csv'
validate_uri = f's3://{bucket}/{prefix}/final/validate/validate_feats.csv'


tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")

training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge")
    
train_data = ParameterString(
    name="TrainData",
    default_value=train_uri
)
validate_data = ParameterString(
    name="ValidateData",
    default_value=validate_uri
)

model_approval_status = ParameterString(
    name='ModelApprovalStatus', default_value='PendingManualApproval')

image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.2-2',
    py_version='py3',
    instance_type='ml.m5.xlarge')

xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    role=role,
    disable_profiler=True)

xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='binary:logistic',
    num_round=25)

step_train = TrainingStep(
    name='train_model',
    estimator=xgb_estimator,
    inputs={
        'train':TrainingInput(
            s3_data=train_data,
            content_type='text/csv'),
        'validation':TrainingInput(
            s3_data=validate_data,
            content_type='text/csv')
            })

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        train_data,
        validate_data
    ],
    steps=[step_train])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(execution_display_name="InsPrebuiltModel3")

In [ ]:
#Train and evaluate
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
prefix = '1_ins_dataset'

pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance-Co-Example"  # Model name in model registry
framework_version = "0.23-1"

train_uri = f's3://{bucket}/{prefix}/final/train/train_feats.csv'
validate_uri = f's3://{bucket}/{prefix}/final/validate/validate_feats.csv'
test_uri = f's3://{bucket}/{prefix}/final/test/test_feats.csv'

tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")

training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge")
    
train_data = ParameterString(
    name="TrainData",
    default_value=train_uri
)
validate_data = ParameterString(
    name="ValidateData",
    default_value=validate_uri
)
test_data = ParameterString(
    name="TestData",
    default_value=test_uri
)

model_approval_status = ParameterString(
    name='ModelApprovalStatus',
    default_value='PendingManualApproval'
)

image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.2-2',
    py_version='py3',
    instance_type='ml.m5.xlarge')

xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    role=role,
    disable_profiler=True,
    output_path=Join(
        on="/",
        values=[
            "s3://{}".format(bucket),
            prefix,
            ExecutionVariables.PIPELINE_EXECUTION_ID,
            "model"],
            ))

xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='binary:logistic',
    num_round=25)

step_train = TrainingStep(
    name='train_model',
    estimator=xgb_estimator,
    inputs={
        'train':TrainingInput(
            s3_data=train_data,
            content_type='text/csv'),
        'validation':TrainingInput(
            s3_data=validate_data,
            content_type='text/csv')
            })

evaluate_model_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    role=role,
    base_job_name="ins-example-job")

evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='evaluation',
    path='evaluation.json')

step_evaluate = ProcessingStep(
    name='evaluate_model',
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_data,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/evaluation',
            destination=Join(
                on='/',
                values=[
                    's3://{}'.format(bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    'evaluation-report']
            )
        )
    ],
    code='evaluate_extd.py',
    property_files=[evaluation_report]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on='/',
            values=[
                step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]['S3Output']['S3Uri'],
                'evaluation.json']
        ),
        content_type='application/json')
)

step_register = RegisterModel(
    name='register-model',
    estimator=xgb_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=['text/csv'],
    response_types=['text/csv'],
    inference_instances=['ml.t2.medium', 'ml.m5.xlarge', 'ml.m5.large'],
    transform_instances=['ml.m5.xlarge'],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        train_data,
        validate_data,
        test_data,
        model_approval_status
    ],
    steps=[step_train, step_evaluate, step_register])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(execution_display_name="InsPrebuiltModelEvalExtd-8")

In [ ]:
#Train, evaluate, and hyperparameter tune
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter
from sagemaker.workflow.steps import TuningStep
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
prefix = '1_ins_dataset'

pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance-Co-Example"  # Model name in model registry
framework_version = "0.23-1"

train_uri = f's3://{bucket}/{prefix}/final/train/train_feats.csv'
validate_uri = f's3://{bucket}/{prefix}/final/validate/validate_feats.csv'
test_uri = f's3://{bucket}/{prefix}/final/test/test_feats.csv'

tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")

training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge")

max_training_jobs = ParameterInteger(name='MaximumTrainingJobs', default_value=1)
max_parallel_training_jobs = ParameterInteger('MaxParallelTrainingJobs', default_value=1)
    
train_data = ParameterString(
    name="TrainData",
    default_value=train_uri
)
validate_data = ParameterString(
    name="ValidateData",
    default_value=validate_uri
)
test_data = ParameterString(
    name="TestData",
    default_value=test_uri
)

model_approval_status = ParameterString(
    name='ModelApprovalStatus',
    default_value='PendingManualApproval'
)

image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.2-2',
    py_version='py3',
    instance_type='ml.m5.xlarge')

xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    role=role,
    disable_profiler=True,
#     output_path=Join(
#         on="/",
#         values=[
#             "s3://{}".format(bucket),
#             prefix,
#             ExecutionVariables.PIPELINE_EXECUTION_ID,
#             "model"])
)

# xgb_estimator.set_hyperparameters(
#     max_depth=5,
#     eta=0.2,
#     gamma=4,
#     min_child_weight=6,
#     subsample=0.8,
#     objective='binary:logistic',
#     num_round=25)

xgb_tuner = HyperparameterTuner(
    estimator=xgb_estimator,
    objective_metric_name="validation:f1",
    hyperparameter_ranges={
        'max_depth': IntegerParameter(1, 10),
        'eta': ContinuousParameter(0, 0.5),
        'gamma': ContinuousParameter(0, 5),
        'min_child_weight': ContinuousParameter(1, 120),
        'num_round': IntegerParameter(1, 2000)
    },
    max_jobs=max_training_jobs,
    max_parallel_jobs=max_parallel_training_jobs)

# xgb_tuner = HyperparameterTuner(
#     estimator=xgb_estimator,
#     objective_metric_name="validation:auc",
#     hyperparameter_ranges={
#         'eta': ContinuousParameter(0, 0.5),
#         'alpha': ContinuousParameter(0, 1000),
#         'min_child_weight': ContinuousParameter(1, 120),
#         'max_depth': IntegerParameter(1, 10),
#         'num_round': IntegerParameter(1, 2000),
#         'subsample': ContinuousParameter(0.5, 1)
#     },
#     max_jobs=max_training_jobs,
#     max_parallel_jobs=max_parallel_training_jobs)

# step_train = TrainingStep(
#     name='train_model',
# #     estimator=xgb_estimator,
#     inputs={
#         'train':TrainingInput(
#             s3_data=train_data,
#             content_type='text/csv'),
#         'validation':TrainingInput(
#             s3_data=validate_data,
#             content_type='text/csv')
#             })

step_tune = TuningStep(
    name='train-tune-model',
    tuner=xgb_tuner,
    inputs={
        'train':TrainingInput(
            s3_data=train_data,
            content_type='text/csv'),
        'validation':TrainingInput(
            s3_data=validate_data,
            content_type='text/csv')})

evaluate_model_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    role=role,
    base_job_name="ins-example-job")

evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='evaluation',
    path='evaluation.json')

step_evaluate = ProcessingStep(
    name='evaluate_model',
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_tune.get_top_model_s3_uri(top_k=0, s3_bucket=bucket),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_data,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/evaluation',
            destination=Join(
                on='/',
                values=[
                    's3://{}'.format(bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    'evaluation-report']
            )
        )
    ],
    code='evaluate_extd.py',
    property_files=[evaluation_report]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on='/',
            values=[
                step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]['S3Output']['S3Uri'],
                'evaluation.json']
        ),
        content_type='application/json')
)

step_register = RegisterModel(
    name='register-model',
    estimator=xgb_estimator,
    model_data=step_tune.get_top_model_s3_uri(top_k=0, s3_bucket=bucket),
    content_types=['text/csv'],
    response_types=['text/csv'],
    inference_instances=['ml.t2.medium', 'ml.m5.xlarge', 'ml.m5.large'],
    transform_instances=['ml.m5.xlarge'],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        max_training_jobs,
        max_parallel_training_jobs,
        train_data,
        validate_data,
        test_data,
        model_approval_status
    ],
    steps=[
#         step_train,
        step_tune,
        step_evaluate,
        step_register])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(execution_display_name="InsPrebuiltModelTune-4")

# Train, Evaluate, Register Prebuilt Model

In [ ]:
import sagemaker
import sagemaker.session

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
prefix = '1_ins_dataset'

pipeline_name = "InsExample"  # SageMaker Pipeline name
model_package_group_name = "Insurance-Co-Example"  # Model name in model registry
framework_version = "0.23-1"

train_uri = f's3://{bucket}/{prefix}/final/train/train_feats.csv'
validate_uri = f's3://{bucket}/{prefix}/final/validate/validate_feats.csv'
test_uri = f's3://{bucket}/{prefix}/final/test/test_feats.csv'

tags = [
    {"Key": "DATASET", "Value": "InsCOIL"},
    {"Key": "SOURCE", "Value": "UCI"}
   ]

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.t3.medium")

training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge")
    
train_data = ParameterString(
    name="TrainData",
    default_value=train_uri
)
validate_data = ParameterString(
    name="ValidateData",
    default_value=validate_uri
)
test_data = ParameterString(
    name="TestData",
    default_value=test_uri
)

model_approval_status = ParameterString(
    name='ModelApprovalStatus',
    default_value='PendingManualApproval'
)

image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.2-2',
    py_version='py3',
    instance_type='ml.m5.xlarge')

xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    role=role,
    disable_profiler=True,
    output_path=Join(
        on="/",
        values=[
            "s3://{}".format(bucket),
            prefix,
            ExecutionVariables.PIPELINE_EXECUTION_ID,
            "model"],
            ))

xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='binary:logistic',
    num_round=25)

step_train = TrainingStep(
    name='train_model',
    estimator=xgb_estimator,
    inputs={
        'train':TrainingInput(
            s3_data=train_data,
            content_type='text/csv'),
        'validation':TrainingInput(
            s3_data=validate_data,
            content_type='text/csv')
            })

evaluate_model_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    role=role,
    base_job_name="ins-example-job")

evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='evaluation',
    path='evaluation.json')

step_evaluate = ProcessingStep(
    name='evaluate_model',
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_data,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/evaluation',
            destination=Join(
                on='/',
                values=[
                    's3://{}'.format(bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    'evaluation-report']
            )
        )
    ],
    code='evaluate.py',
    property_files=[evaluation_report]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on='/',
            values=[
                step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]['S3Output']['S3Uri'],
                'evaluation.json']
        ),
        content_type='application/json')
)

step_register = RegisterModel(
    name='register-model',
    estimator=xgb_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=['text/csv'],
    response_types=['text/csv'],
    inference_instances=['ml.t2.medium', 'ml.m5.xlarge', 'ml.m5.large'],
    transform_instances=['ml.m5.xlarge'],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        train_data,
        validate_data,
        test_data,
        model_approval_status
    ],
    steps=[step_train, step_evaluate, step_register])

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(execution_display_name="InsPrebuiltModelEval-7")